In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [10]:
import ee
import pandas as pd
import numpy as np
import datetime
import os
from dateutil.relativedelta import relativedelta
from google.colab import files
import matplotlib.pyplot as plt
import folium
import csv
import math
import calendar


### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gen-lang-client-0253961861')

In [5]:
def get_average_albedo(region, start_date, end_date, platform, albedo_var, scale):
    """
    Extracts the average numerical albedo value from Google Earth Engine.

    Parameters:
    - region: dict, region definition in GeoJSON format.
    - start_date: str, start date in 'YYYY-MM-DD' format.
    - end_date: str, end date in 'YYYY-MM-DD' format.
    - platform: str, the ID of the Earth Engine image collection.
    - albedo_var: str, the name of the albedo band to extract.
    - scale: int, scale in meters.

    Returns:
    - Average numerical albedo value for the specified period and region.
    """
    try:
        # Convert the GeoJSON region to an Earth Engine Geometry
        ee_region = ee.Geometry(region)

        # Get the image collection and filter by date
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Select the albedo band
        albedo_band = collection.select(albedo_var)

        # Apply scaling factor for MODIS datasets (scale to 0-1)
        if 'MODIS' in platform:
            albedo_band = albedo_band.map(lambda img: img.multiply(0.001))

        # Calculate the mean image over the time period
        mean_image = albedo_band.mean()

        # Reduce the region to get the average albedo
        reduction = mean_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Retrieve the result
        average_albedo = reduction.get(albedo_var).getInfo()

        return average_albedo

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Define a region
region = {
    'type': 'Polygon',
    # 'coordinates': [[
    # [-5.962680158041947,37.28724279163582],
    # [-5.963514434594459,37.29055505268988],
    # [-5.969837348562913,37.28838549498776],
    # [-5.968610951356155,37.28565311917788],
    # [-5.962680158041947,37.28724279163582]
    # ]]

    # Guillena
#     'coordinates': [[
# [-6.026331827726255,37.58208818356712],
# [-5.985282249082786,37.58608611336135],
# [-5.987439073055367,37.59466234623054],
# [-5.989702713054932,37.60300770489669],
# [-6.028373414514872,37.59587639387659],
# [-6.026331827726255,37.58208818356712],

#     ]]

# Bela Bela
    'coordinates': [[
[28.36849802101698,-24.86729867556011],
[28.39048392044633,-24.86524899949362],
[28.39110470039546,-24.86045795568642],
[28.36849802101698,-24.86729867556011],
    ]]

}

# Example parameters for MODIS (updated dataset ID)
modis_params = {
    'start_date': '2009-01-01',
    'end_date': '2019-12-31',
    'platform': 'MODIS/061/MCD43A3',  # Updated to v061
    'albedo_var': 'Albedo_BSA_shortwave', # 'Albedo_BSA_vis',  # Black-sky albedo (visible)
    'scale': 500
}

# Example parameters for ERA5-Land (using forecast_albedo)
era5_params = {
    'start_date': '2009-01-01',
    'end_date': '2019-12-31',
    'platform': 'ECMWF/ERA5_LAND/DAILY_AGGR',
    'albedo_var': 'forecast_albedo',  # Correct band name
    'scale': 11132  # 0.1 degree ≈ 11,132 meters
}

# Compute and print albedo for MODIS
modis_albedo = get_average_albedo(region, **modis_params)
print(f"MODIS Average Albedo: {modis_albedo}")

# Compute and print albedo for ERA5-Land
era5_albedo = get_average_albedo(region, **era5_params)
print(f"ERA5-Land Average Albedo: {era5_albedo}")

MODIS Average Albedo: 0.12032884994320267
ERA5-Land Average Albedo: None


In [6]:
# Función para extraer albedo promedio
def get_average_albedo(region, start_date, end_date, platform='MODIS/061/MCD43A3',
                      albedo_var='Albedo_BSA_shortwave', scale=500):
    """
    Extrae el valor promedio de albedo numérico desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - start_date: str, fecha de inicio en formato 'YYYY-MM-DD'.
    - end_date: str, fecha de fin en formato 'YYYY-MM-DD'.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
    - albedo_var: str, el nombre de la banda de albedo a extraer.
    - scale: int, escala en metros.

    Retorna:
    - Valor promedio de albedo numérico para el período y región especificados.
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Obtener la colección de imágenes y filtrar por fecha
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Seleccionar la banda de albedo
        albedo_band = collection.select(albedo_var)

        # Aplicar factor de escala para conjuntos de datos MODIS (escalar a 0-1)
        if 'MODIS' in platform:
            albedo_band = albedo_band.map(lambda img: img.multiply(0.001))

        # Calcular la imagen media durante el período de tiempo
        mean_image = albedo_band.mean()

        # Reducir la región para obtener el albedo promedio
        reduction = mean_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Recuperar el resultado
        average_albedo = reduction.get(albedo_var).getInfo()

        return average_albedo

    except Exception as e:
        print(f"Error al extraer el valor promedio de albedo: {e}")
        return None

# Función para extraer radiación solar promedio
def get_average_solar_radiation(region, start_date, end_date, platform='MODIS/061/MOD11A1',
                               radiation_var='LST_Day_1km', scale=1000):
    """
    Extrae el valor promedio de radiación solar desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - start_date: str, fecha de inicio en formato 'YYYY-MM-DD'.
    - end_date: str, fecha de fin en formato 'YYYY-MM-DD'.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
      Nota: Se usa MOD11A1 como alternativa ya que MCD18A1 está obsoleto.
    - radiation_var: str, el nombre de la banda de radiación a extraer.
    - scale: int, escala en metros.

    Retorna:
    - Valor promedio de radiación solar (W/m²) para el período y región especificados.
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Obtener la colección de imágenes y filtrar por fecha
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Verificar si hay imágenes disponibles
        count = collection.size().getInfo()
        if count == 0:
            print(f"No hay imágenes de radiación solar disponibles para el período {start_date} a {end_date}")
            return None

        # Verificar si la banda solicitada está disponible
        first_image = collection.first()
        available_bands = first_image.bandNames().getInfo()

        if radiation_var not in available_bands:
            print(f"La banda {radiation_var} no está disponible. Bandas disponibles: {available_bands}")
            # Intentar encontrar una banda alternativa
            if 'LST_Day_1km' in available_bands:
                radiation_var = 'LST_Day_1km'
                print(f"Usando banda alternativa: {radiation_var}")
            else:
                print("No se encontraron bandas de radiación solar adecuadas")
                return None

        # Seleccionar la banda de radiación
        radiation_band = collection.select(radiation_var)

        # Calcular la imagen media durante el período de tiempo
        mean_image = radiation_band.mean()

        # Reducir la región para obtener la radiación promedio
        reduction = mean_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Recuperar el resultado
        average_radiation = reduction.get(radiation_var).getInfo()

        # Convertir a W/m² (aproximación basada en temperatura)
        if average_radiation is not None:
            # Convertir temperatura a radiación (aproximación)
            # Fórmula simplificada: R = σT⁴, donde σ es la constante de Stefan-Boltzmann
            # T está en Kelvin, y se multiplica por 0.02 para MODIS LST
            temp_kelvin = average_radiation * 0.02
            stefan_boltzmann = 5.67e-8  # W/(m²·K⁴)
            average_radiation = stefan_boltzmann * (temp_kelvin ** 4)

        return average_radiation

    except Exception as e:
        print(f"Error al extraer el valor promedio de radiación solar: {e}")
        return None

# Función para extraer temperatura superficial promedio
def get_average_temperature(region, start_date, end_date, platform='MODIS/061/MOD11A1',
                           temp_var='LST_Day_1km', scale=1000):
    """
    Extrae el valor promedio de temperatura superficial desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - start_date: str, fecha de inicio en formato 'YYYY-MM-DD'.
    - end_date: str, fecha de fin en formato 'YYYY-MM-DD'.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
    - temp_var: str, el nombre de la banda de temperatura a extraer.
    - scale: int, escala en metros.

    Retorna:
    - Valor promedio de temperatura (°C) para el período y región especificados.
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Obtener la colección de imágenes y filtrar por fecha
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Seleccionar la banda de temperatura
        temp_band = collection.select(temp_var)

        # Calcular la imagen media durante el período de tiempo
        mean_image = temp_band.mean()

        # Reducir la región para obtener la temperatura promedio
        reduction = mean_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Recuperar el resultado
        average_temp = reduction.get(temp_var).getInfo()

        # Convertir a Celsius (los valores de MODIS están en Kelvin * 0.02)
        if 'MODIS' in platform and average_temp is not None:
            average_temp = (average_temp * 0.02) - 273.15

        return average_temp

    except Exception as e:
        print(f"Error al extraer el valor promedio de temperatura: {e}")
        return None

# Función para extraer velocidad y dirección del viento promedio
def get_average_wind(region, start_date, end_date, platform='ECMWF/ERA5_LAND/HOURLY', scale=10000):
    """
    Extrae los valores promedio de velocidad y dirección del viento desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - start_date: str, fecha de inicio en formato 'YYYY-MM-DD'.
    - end_date: str, fecha de fin en formato 'YYYY-MM-DD'.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
    - scale: int, escala en metros.

    Retorna:
    - dict con valores promedio de velocidad (m/s) y dirección (grados) del viento.
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Obtener la colección de imágenes y filtrar por fecha
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Seleccionar las bandas de componentes del viento
        u_band = 'u_component_of_wind_10m'
        v_band = 'v_component_of_wind_10m'

        # Verificar si las bandas están disponibles
        first_image = collection.first()
        available_bands = first_image.bandNames().getInfo()

        if u_band not in available_bands or v_band not in available_bands:
            print(f"Las bandas de viento no están disponibles. Bandas disponibles: {available_bands}")
            return None

        # Seleccionar las bandas de componentes del viento
        wind_band = collection.select([u_band, v_band])

        # Calcular la imagen media durante el período de tiempo
        mean_image = wind_band.mean()

        # Reducir la región para obtener los componentes promedio
        reduction = mean_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Recuperar los resultados
        u_component = reduction.get(u_band).getInfo()
        v_component = reduction.get(v_band).getInfo()

        # Calcular velocidad y dirección
        if u_component is not None and v_component is not None:
            speed = math.sqrt(u_component**2 + v_component**2)
            direction = (270 - math.degrees(math.atan2(v_component, u_component))) % 360
            return {
                'speed': speed,
                'direction': direction
            }
        else:
            return None

    except Exception as e:
        print(f"Error al extraer los valores promedio de viento: {e}")
        return None

# Función para extraer elevación y variables topográficas
def get_average_topography(region, platform='USGS/SRTMGL1_003', scale=30):
    """
    Extrae los valores promedio de elevación y variables topográficas derivadas desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
    - scale: int, escala en metros.

    Retorna:
    - dict con valores promedio de elevación (m), pendiente (grados) y aspecto (grados).
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Cargar el modelo de elevación
        elevation = ee.Image(platform).select('elevation')

        # Calcular pendiente y aspecto
        slope = ee.Terrain.slope(elevation)
        aspect = ee.Terrain.aspect(elevation)

        # Combinar en una sola imagen
        topo = elevation.addBands(slope).addBands(aspect).rename(['elevation', 'slope', 'aspect'])

        # Reducir la región para obtener los valores promedio
        reduction = topo.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Recuperar los resultados
        elevation_value = reduction.get('elevation').getInfo()
        slope_value = reduction.get('slope').getInfo()
        aspect_value = reduction.get('aspect').getInfo()

        return {
            'elevation': elevation_value,
            'slope': slope_value,
            'aspect': aspect_value
        }

    except Exception as e:
        print(f"Error al extraer los valores de topografía: {e}")
        return None

# Función para extraer cobertura terrestre
def get_landcover_distribution(region, year, platform='MODIS/061/MCD12Q1',
                              lc_var='LC_Type1', scale=500):
    """
    Extrae la distribución de tipos de cobertura terrestre desde Google Earth Engine.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - year: int, año para el cual obtener la cobertura terrestre.
    - platform: str, el ID de la colección de imágenes de Earth Engine.
    - lc_var: str, tipo de clasificación de cobertura terrestre.
    - scale: int, escala en metros.

    Retorna:
    - dict con la distribución porcentual de tipos de cobertura terrestre.
    """
    try:
        # Convertir la región GeoJSON a una geometría de Earth Engine
        ee_region = ee.Geometry(region)

        # Definir el rango de fechas para el año especificado
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"

        # Obtener la colección de imágenes y filtrar por fecha
        collection = ee.ImageCollection(platform).filterDate(start_date, end_date)

        # Verificar si hay imágenes disponibles
        count = collection.size().getInfo()
        if count == 0:
            print(f"No hay imágenes de cobertura terrestre disponibles para el año {year}")
            # Intentar con años anteriores
            for prev_year in range(year-1, year-5, -1):
                print(f"Intentando con datos de cobertura terrestre del año {prev_year}...")
                prev_start_date = f"{prev_year}-01-01"
                prev_end_date = f"{prev_year}-12-31"
                collection = ee.ImageCollection(platform) \
                    .filterDate(prev_start_date, prev_end_date)
                count = collection.size().getInfo()
                if count > 0:
                    print(f"Se encontraron datos de cobertura terrestre para el año {prev_year}")
                    break

            if count == 0:
                return None

        # Obtener la primera imagen (generalmente hay una por año)
        landcover = collection.first().select(lc_var)

        # Calcular el histograma de clases de cobertura terrestre
        histogram = landcover.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10
        )

        # Obtener la distribución
        distribution = histogram.get(lc_var).getInfo()

        # Convertir claves a enteros y calcular porcentajes
        total_pixels = sum(distribution.values())
        result = {}

        for class_id, count in distribution.items():
            class_id_int = int(class_id)
            percentage = (count / total_pixels) * 100
            result[class_id_int] = percentage

        return result

    except Exception as e:
        print(f"Error al extraer la distribución de cobertura terrestre: {e}")
        return None

# Ejemplo de uso:
if __name__ == "__main__":

    # Definir una región de ejemplo (polígono en Sudáfrica)
    region_example = {
        'type': 'Polygon',
        'coordinates': [[
            [28.36849802101698, -24.86729867556011],
            [28.39048392044633, -24.86524899949362],
            [28.39110470039546, -24.86045795568642],
            [28.36849802101698, -24.86729867556011]
        ]]
    }

    # Parámetros para el período de tiempo
    start_date = '2018-01-01'
    end_date = '2018-12-31'

    # Extraer albedo
    albedo_bsa = get_average_albedo(
        region=region_example,
        start_date=start_date,
        end_date=end_date,
        albedo_var='Albedo_BSA_shortwave'
    )

    albedo_wsa = get_average_albedo(
        region=region_example,
        start_date=start_date,
        end_date=end_date,
        albedo_var='Albedo_WSA_shortwave'
    )

    print(f"Albedo Black-Sky promedio: {albedo_bsa}")
    print(f"Albedo White-Sky promedio: {albedo_wsa}")

    # Extraer radiación solar (usando MOD11A1 como alternativa)
    radiation = get_average_solar_radiation(
        region=region_example,
        start_date=start_date,
        end_date=end_date
    )

    print(f"Radiación solar promedio: {radiation} W/m²")

    # Extraer temperatura
    temperature_day = get_average_temperature(
        region=region_example,
        start_date=start_date,
        end_date=end_date,
        temp_var='LST_Day_1km'
    )

    temperature_night = get_average_temperature(
        region=region_example,
        start_date=start_date,
        end_date=end_date,
        temp_var='LST_Night_1km'
    )

    print(f"Temperatura diurna promedio: {temperature_day} °C")
    print(f"Temperatura nocturna promedio: {temperature_night} °C")

    # Extraer viento
    wind = get_average_wind(
        region=region_example,
        start_date=start_date,
        end_date=end_date
    )

    if wind:
        print(f"Velocidad del viento promedio: {wind['speed']} m/s")
        print(f"Dirección del viento promedio: {wind['direction']} grados")

    # Extraer topografía
    topo = get_average_topography(region=region_example)

    if topo:
        print(f"Elevación promedio: {topo['elevation']} m")
        print(f"Pendiente promedio: {topo['slope']} grados")
        print(f"Aspecto promedio: {topo['aspect']} grados")

    # Extraer cobertura terrestre
    landcover = get_landcover_distribution(
        region=region_example,
        year=2018
    )

    if landcover:
        print("Distribución de cobertura terrestre (%):")
        for class_id, percentage in landcover.items():
            print(f"  Clase {class_id}: {percentage:.2f}%")


Albedo Black-Sky promedio: 0.12033904627482594
Albedo White-Sky promedio: 0.12920445609436437
Radiación solar promedio: 474.99377398173266 W/m²
Temperatura diurna promedio: 29.38541067079393 °C
Temperatura nocturna promedio: 12.641236709959117 °C
Error al extraer los valores promedio de viento: name 'math' is not defined
Elevación promedio: 1206.2576803674967 m
Pendiente promedio: 1.6981001570742085 grados
Aspecto promedio: 155.24301917526793 grados
Distribución de cobertura terrestre (%):
  Clase 10: 100.00%


CSV monthly

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Script para extraer valores mensuales de variables relevantes para plantas solares fotovoltaicas
desde Google Earth Engine para un polígono definido y exportarlos a CSV.

Variables incluidas:
- Albedo de superficie (black-sky y white-sky)
- Radiación solar
- Temperatura superficial
- Velocidad y dirección del viento
- Elevación y variables topográficas derivadas
- Cobertura terrestre

"""


def generate_monthly_dates(start_year, start_month, end_year, end_month):
    """
    Genera una lista de tuplas con fechas de inicio y fin para cada mes en el rango especificado.

    Parámetros:
    - start_year: int, año de inicio
    - start_month: int, mes de inicio (1-12)
    - end_year: int, año de fin
    - end_month: int, mes de fin (1-12)

    Retorna:
    - Lista de tuplas (fecha_inicio, fecha_fin, etiqueta_mes) para cada mes
    """
    monthly_dates = []

    # Fecha inicial
    current_date = datetime.datetime(start_year, start_month, 1)

    # Fecha final
    end_date = datetime.datetime(end_year, end_month, 1)

    # Generar fechas mensuales
    while current_date <= end_date:
        # Primer día del mes
        first_day = current_date.replace(day=1)

        # Último día del mes
        last_day = current_date.replace(day=calendar.monthrange(current_date.year, current_date.month)[1])

        # Formato de fechas para Earth Engine
        start_str = first_day.strftime('%Y-%m-%d')
        end_str = last_day.strftime('%Y-%m-%d')

        # Etiqueta del mes (YYYY-MM)
        month_label = current_date.strftime('%Y-%m')

        # Añadir a la lista
        monthly_dates.append((start_str, end_str, month_label))

        # Avanzar al siguiente mes
        current_date = current_date + relativedelta(months=1)

    return monthly_dates

def extract_monthly_variables(region, start_year, start_month, end_year, end_month, output_csv):
    """
    Extrae valores mensuales de todas las variables para un polígono y los guarda en un archivo CSV.

    Parámetros:
    - region: dict, definición de la región en formato GeoJSON.
    - start_year: int, año de inicio
    - start_month: int, mes de inicio (1-12)
    - end_year: int, año de fin
    - end_month: int, mes de fin (1-12)
    - output_csv: str, ruta del archivo CSV de salida

    Retorna:
    - bool: True si la extracción fue exitosa, False en caso contrario
    """
    try:
        # Generar fechas mensuales
        monthly_dates = generate_monthly_dates(start_year, start_month, end_year, end_month)

        # Obtener datos topográficos (constantes para todos los meses)
        print("Extrayendo datos topográficos...")
        topo_data = get_average_topography(region)

        if not topo_data:
            print("⚠️ No se pudieron obtener datos topográficos.")
            topo_data = {
                'elevation': None,
                'slope': None,
                'aspect': None
            }

        # Preparar encabezados del CSV
        headers = [
            'Fecha',
            'Albedo_BSA',
            'Albedo_WSA',
            'Radiacion_Solar',
            'Temperatura_Dia',
            'Temperatura_Noche',
            'Viento_Velocidad',
            'Viento_Direccion',
            'Elevacion',
            'Pendiente',
            'Aspecto',
            'Cobertura_Terrestre_Principal',
            'Cobertura_Terrestre_Porcentaje'
        ]

        # Crear archivo CSV y escribir encabezados
        with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)

            # Procesar cada mes
            total_months = len(monthly_dates)
            for i, (start_date, end_date, month_label) in enumerate(monthly_dates):
                print(f"Procesando mes {i+1}/{total_months}: {month_label}...")

                # Inicializar fila con valores por defecto
                row = [month_label, None, None, None, None, None, None, None,
                       topo_data['elevation'], topo_data['slope'], topo_data['aspect'], None, None]

                # Extraer albedo BSA
                try:
                    albedo_bsa = get_average_albedo(
                        region=region,
                        start_date=start_date,
                        end_date=end_date,
                        albedo_var='Albedo_BSA_shortwave'
                    )
                    row[1] = albedo_bsa
                except Exception as e:
                    print(f"Error al extraer albedo BSA para {month_label}: {e}")

                # Extraer albedo WSA
                try:
                    albedo_wsa = get_average_albedo(
                        region=region,
                        start_date=start_date,
                        end_date=end_date,
                        albedo_var='Albedo_WSA_shortwave'
                    )
                    row[2] = albedo_wsa
                except Exception as e:
                    print(f"Error al extraer albedo WSA para {month_label}: {e}")

                # Extraer radiación solar
                try:
                    radiation = get_average_solar_radiation(
                        region=region,
                        start_date=start_date,
                        end_date=end_date
                    )
                    row[3] = radiation
                except Exception as e:
                    print(f"Error al extraer radiación solar para {month_label}: {e}")

                # Extraer temperatura diurna
                try:
                    temp_day = get_average_temperature(
                        region=region,
                        start_date=start_date,
                        end_date=end_date,
                        temp_var='LST_Day_1km'
                    )
                    row[4] = temp_day
                except Exception as e:
                    print(f"Error al extraer temperatura diurna para {month_label}: {e}")

                # Extraer temperatura nocturna
                try:
                    temp_night = get_average_temperature(
                        region=region,
                        start_date=start_date,
                        end_date=end_date,
                        temp_var='LST_Night_1km'
                    )
                    row[5] = temp_night
                except Exception as e:
                    print(f"Error al extraer temperatura nocturna para {month_label}: {e}")

                # Extraer viento
                try:
                    wind = get_average_wind(
                        region=region,
                        start_date=start_date,
                        end_date=end_date
                    )
                    if wind:
                        row[6] = wind['speed']
                        row[7] = wind['direction']
                except Exception as e:
                    print(f"Error al extraer viento para {month_label}: {e}")

                # Extraer cobertura terrestre (solo para el primer mes de cada año)
                if i == 0 or month_label.endswith('-01'):
                    year = int(month_label.split('-')[0])
                    try:
                        landcover = get_landcover_distribution(
                            region=region,
                            year=year
                        )
                        if landcover:
                            # Encontrar la clase principal (mayor porcentaje)
                            main_class = max(landcover.items(), key=lambda x: x[1])
                            row[11] = main_class[0]  # Clase
                            row[12] = main_class[1]  # Porcentaje
                    except Exception as e:
                        print(f"Error al extraer cobertura terrestre para {year}: {e}")

                # Escribir fila en el CSV
                writer.writerow(row)

                # Mostrar progreso
                print(f"✅ Mes {month_label} procesado")

        print(f"\n✅ Extracción completada. Datos guardados en {output_csv}")
        return True

    except Exception as e:
        print(f"❌ Error durante la extracción mensual: {e}")
        return False



In [13]:
# Ejemplo de uso:
if __name__ == "__main__":

    # Definir una región de ejemplo (polígono en Sudáfrica)
    region_example = {
        'type': 'Polygon',
        'coordinates': [[
            [28.28563380172433,-24.80764984260767],
            [28.29998800378916,-24.92613583264521],
            [28.50667618936926,-24.8824036035049],
            [28.4679000955403,-24.76723542098093],
            [28.28563380172433,-24.80764984260767],
        ]]
    }




    # Extraer datos mensuales para un año
    extract_monthly_variables(
        region=region_example,
        start_year=2008,
        start_month=1,
        end_year=2018,
        end_month=12,
        output_csv='solar_variables_monthly_2018.csv'
    )

    print(f"\nPuedes encontrar el archivo CSV en: {os.path.abspath('solar_variables_monthly_2008_2018.csv')}")


Extrayendo datos topográficos...
Procesando mes 1/132: 2008-01...
✅ Mes 2008-01 procesado
Procesando mes 2/132: 2008-02...
✅ Mes 2008-02 procesado
Procesando mes 3/132: 2008-03...
✅ Mes 2008-03 procesado
Procesando mes 4/132: 2008-04...
✅ Mes 2008-04 procesado
Procesando mes 5/132: 2008-05...
✅ Mes 2008-05 procesado
Procesando mes 6/132: 2008-06...
✅ Mes 2008-06 procesado
Procesando mes 7/132: 2008-07...
✅ Mes 2008-07 procesado
Procesando mes 8/132: 2008-08...
✅ Mes 2008-08 procesado
Procesando mes 9/132: 2008-09...
✅ Mes 2008-09 procesado
Procesando mes 10/132: 2008-10...
✅ Mes 2008-10 procesado
Procesando mes 11/132: 2008-11...
✅ Mes 2008-11 procesado
Procesando mes 12/132: 2008-12...
✅ Mes 2008-12 procesado
Procesando mes 13/132: 2009-01...
✅ Mes 2009-01 procesado
Procesando mes 14/132: 2009-02...
✅ Mes 2009-02 procesado
Procesando mes 15/132: 2009-03...
✅ Mes 2009-03 procesado
Procesando mes 16/132: 2009-04...
✅ Mes 2009-04 procesado
Procesando mes 17/132: 2009-05...
✅ Mes 2009-05 